In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('pyspark - example toPandas()').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/28 11:28:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/02/28 11:28:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
from pyspark.sql.functions import *

In [3]:
df = spark.read.option("header", "true").csv('covid.csv')

In [4]:
df.count()

164015

In [5]:
df.columns

['iso_code',
 'continent',
 'location',
 'date',
 'total_cases',
 'new_cases',
 'new_cases_smoothed',
 'total_deaths',
 'new_deaths',
 'new_deaths_smoothed',
 'total_cases_per_million',
 'new_cases_per_million',
 'new_cases_smoothed_per_million',
 'total_deaths_per_million',
 'new_deaths_per_million',
 'new_deaths_smoothed_per_million',
 'reproduction_rate',
 'icu_patients',
 'icu_patients_per_million',
 'hosp_patients',
 'hosp_patients_per_million',
 'weekly_icu_admissions',
 'weekly_icu_admissions_per_million',
 'weekly_hosp_admissions',
 'weekly_hosp_admissions_per_million',
 'new_tests',
 'population',
 'population_density']

In [6]:
df.dtypes

[('iso_code', 'string'),
 ('continent', 'string'),
 ('location', 'string'),
 ('date', 'string'),
 ('total_cases', 'string'),
 ('new_cases', 'string'),
 ('new_cases_smoothed', 'string'),
 ('total_deaths', 'string'),
 ('new_deaths', 'string'),
 ('new_deaths_smoothed', 'string'),
 ('total_cases_per_million', 'string'),
 ('new_cases_per_million', 'string'),
 ('new_cases_smoothed_per_million', 'string'),
 ('total_deaths_per_million', 'string'),
 ('new_deaths_per_million', 'string'),
 ('new_deaths_smoothed_per_million', 'string'),
 ('reproduction_rate', 'string'),
 ('icu_patients', 'string'),
 ('icu_patients_per_million', 'string'),
 ('hosp_patients', 'string'),
 ('hosp_patients_per_million', 'string'),
 ('weekly_icu_admissions', 'string'),
 ('weekly_icu_admissions_per_million', 'string'),
 ('weekly_hosp_admissions', 'string'),
 ('weekly_hosp_admissions_per_million', 'string'),
 ('new_tests', 'string'),
 ('population', 'string'),
 ('population_density', 'string')]

In [7]:
df.select(col('date')).show(5)

+---------+
|     date|
+---------+
|2/24/2020|
|2/25/2020|
|2/26/2020|
|2/27/2020|
|2/28/2020|
+---------+
only showing top 5 rows



In [8]:
df = df.withColumn('date',when(df.date.like("_/__/____"),concat(lit("0"),df.date))
          .when(df.date.like("_/_/____"),concat(lit("0"),df.date.substr(1,2),lit("0"),df.date.substr(3,1),df.date.substr(4,5)))
          .when(df.date.like("__/_/____"),concat(df.date.substr(1,3),lit("0"),df.date.substr(4,1),df.date.substr(5,5)))
          .otherwise(df.date))

In [9]:
from pyspark.sql.functions import regexp_replace
df = df.withColumn('date', regexp_replace('date', '/', '-')) \
#.show(truncate=False)

In [10]:
df.show(5)

22/02/28 11:29:26 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+---------+-----------+----------+-----------+---------+------------------+------------+----------+-------------------+-----------------------+---------------------+------------------------------+------------------------+----------------------+-------------------------------+-----------------+------------+------------------------+-------------+-------------------------+---------------------+---------------------------------+----------------------+----------------------------------+---------+----------+------------------+
|iso_code|continent|   location|      date|total_cases|new_cases|new_cases_smoothed|total_deaths|new_deaths|new_deaths_smoothed|total_cases_per_million|new_cases_per_million|new_cases_smoothed_per_million|total_deaths_per_million|new_deaths_per_million|new_deaths_smoothed_per_million|reproduction_rate|icu_patients|icu_patients_per_million|hosp_patients|hosp_patients_per_million|weekly_icu_admissions|weekly_icu_admissions_per_million|weekly_hosp_admissions|week

In [11]:
df = df.withColumn(("date"),to_date(col("date"),"MM-dd-yyyy"))

In [13]:
df.toPandas().to_csv("./Clean Data/covid.csv",index = False)